In [1]:
DATASET_PATH = {
    'train': 'workdir/datasets/iris-train-2-s1.csv', 
    'test': 'workdir/datasets/iris-test-2-s1.csv'
}
SAVE_RESULTS_PATH = "workdir/results/res1-new.results"
N_ESTIMATORS = 5
MIN_SAMPLES_SPLIT = 2
N_JOBS = 1
MAX_DEPTH = 5
SUBSPACES = 5
CV = 5
CV_REPEATS = 10
RF_TYPE = 'extraTrees' # 'randomForest'
SELECTION_METHODS = ['balanced_accuracy', 'accuracy', 'rf_accuracy', 'rf_balanced_accuracy', 'accuracy/accuracy_stddev'] # 'balanced_accuracy' # 'accuracy', 'f1_weighted'
PARALLELISM = "loky"

In [2]:
import pickle
import numpy as np
import pandas as pd
from reduction.api import Instance, AdjacentOrNot, Relation, Rule, Features, Statement
from reduction.extract_rules import extract_rules
from reduction.measure_adjacencies import measure_rules
from toolz.curried import pipe, filter, map, reduce
from reduction.utils import bound_rule, join_consecutive_statements
from joblib import Parallel, delayed
from sklearn.metrics import recall_score, make_scorer, confusion_matrix
from sklearn.metrics import accuracy_score, balanced_accuracy_score, precision_score, recall_score, f1_score, make_scorer, confusion_matrix
from imblearn.metrics import geometric_mean_score

In [3]:
from dask.distributed import Client
import joblib

if PARALLELISM == 'dask':
    parallel_backend = 'dask'
    client = Client("localhost:56436")
else:
    parallel_backend = 'loky'

## LOAD

In [4]:
train_data = pd.read_csv(DATASET_PATH['train'])
test_data = pd.read_csv(DATASET_PATH['test'])

In [5]:
x_train = train_data.drop('TARGET', axis=1).values
y_train = train_data['TARGET'].values
x_test = test_data.drop('TARGET', axis=1).values
y_test = test_data['TARGET'].values

In [6]:
meta = {
        **DATASET_PATH,
       }

In [7]:
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from reduction.classification import measure_acc, to_instance, RulesClassifier, measure_metric

In [8]:
if RF_TYPE == 'randomForest':
    clf_rf = RandomForestClassifier(max_depth=MAX_DEPTH, n_estimators=N_ESTIMATORS, min_samples_split=MIN_SAMPLES_SPLIT, random_state=42)
elif RF_TYPE == 'extraTrees':
    clf_rf = ExtraTreesClassifier(max_depth=MAX_DEPTH, n_estimators=N_ESTIMATORS, min_samples_split=MIN_SAMPLES_SPLIT, random_state=42)
clf_rf.fit(x_train, y_train)

ExtraTreesClassifier(max_depth=5, n_estimators=5, random_state=42)

In [9]:
clf_dt = DecisionTreeClassifier(random_state=42, min_samples_split=MIN_SAMPLES_SPLIT, max_depth=MAX_DEPTH)
clf_dt.fit(x_train, y_train)

DecisionTreeClassifier(max_depth=5, random_state=42)

In [10]:
rf_test_predict = clf_rf.predict(x_test)
dt_test_predict = clf_dt.predict(x_test)

In [11]:
meta = {
    **meta,
    "DT_test_acc": accuracy_score(y_test, dt_test_predict),
    "RF_test_acc": accuracy_score(y_test, rf_test_predict),
    "DT_test_confusion_matrix": confusion_matrix(y_test, dt_test_predict),
    "RF_test_confusion_matrix": confusion_matrix(y_test, rf_test_predict),
    "RF_test_f1": f1_score(y_test, rf_test_predict, average='weighted'),
    "DT_test_f1": f1_score(y_test, dt_test_predict, average='weighted'),
    "DT_test_gmean": geometric_mean_score(y_test, dt_test_predict, average='weighted'),
    "RF_test_gmean": geometric_mean_score(y_test, rf_test_predict, average='weighted'),
    "DT_test_balanced_accuracy": balanced_accuracy_score(y_test, dt_test_predict),
    "RF_test_balanced_accuracy": balanced_accuracy_score(y_test, rf_test_predict),
    "x_test": x_test,
    "RF_test_predictions": rf_test_predict,
    "DT_test_predictions": dt_test_predict,
    "SELECTION_METHODS": SELECTION_METHODS,
    "CV": CV,
    "RF_TYPE": RF_TYPE,
    "CV_REPEATS": CV_REPEATS
}

In [12]:
%%time

all_rules = pipe(
    clf_rf.estimators_,
    map(lambda estimator: extract_rules(estimator)),
    reduce(set.union),
    map(lambda r: join_consecutive_statements(r)),
#     map(lambda r: bound_rule(r, x_train)),
    list
)


CPU times: user 39.1 ms, sys: 0 ns, total: 39.1 ms
Wall time: 37.9 ms


In [13]:
all_statements = pipe(
    all_rules,
    set,
    map(lambda r: list(r.statements)),
    reduce(list.__add__),
)

# Find all non-overlapping rule cliques

In [14]:
import networkx as nx
from networkx.algorithms.clique import enumerate_all_cliques, find_cliques

In [15]:
g = nx.Graph()

In [16]:
%%time

with joblib.parallel_backend(parallel_backend):
    all_rule_measurements = measure_rules(all_rules, n_jobs=N_JOBS)

CPU times: user 69 ms, sys: 1.3 ms, total: 70.3 ms
Wall time: 68.6 ms


In [17]:
def add_to_graph(measurements_tuple, measurement):
    
    if measurement == AdjacentOrNot.NOT_ADJACENT:
        rule_1, rule_2 = measurements_tuple
        rule_idx_1 = all_rules.index(rule_1)
        rule_idx_2 = all_rules.index(rule_2)
        
        g.add_node(rule_idx_1)

        g.add_node(rule_idx_2)

        g.add_edge(rule_idx_1, rule_idx_2)

In [18]:
%%time
for measurements_tuple, measurement in all_rule_measurements.items():
    
    add_to_graph(measurements_tuple, measurement)

CPU times: user 35.3 ms, sys: 904 µs, total: 36.2 ms
Wall time: 34.7 ms


In [19]:
%%time
all_subspaces = list(filter(lambda s: len(s) <= SUBSPACES)(find_cliques(g)))

subspaces_to_check = all_subspaces

for clique_size in reversed(range(1, SUBSPACES + 1)):
    subspaces_by_size_of_param = list(filter(lambda s: len(s) == SUBSPACES)(all_subspaces))
    print(f"clique = {clique_size}")
    if not len(subspaces_by_size_of_param) == 0:
        subspaces_to_check = subspaces_by_size_of_param
        break

clique = 5
CPU times: user 4.6 ms, sys: 0 ns, total: 4.6 ms
Wall time: 4.47 ms


In [20]:
if not subspaces_to_check:
    subspaces_to_check = [[r_idx] for r_idx in list(range(len(all_rules)))]

In [21]:
len(subspaces_to_check)

24

In [22]:
meta = {
    **meta,
    'total_rules' : len(all_rules),
    'total_subspaces_to_check': len(subspaces_to_check),
#     'clique_size': clique_size
}

In [23]:
meta

{'train': 'workdir/datasets/iris-train-2-s1.csv',
 'test': 'workdir/datasets/iris-test-2-s1.csv',
 'DT_test_acc': 0.94,
 'RF_test_acc': 0.96,
 'DT_test_confusion_matrix': array([[17,  0,  0],
        [ 0, 15,  2],
        [ 0,  1, 15]]),
 'RF_test_confusion_matrix': array([[17,  0,  0],
        [ 0, 15,  2],
        [ 0,  0, 16]]),
 'RF_test_f1': 0.9599264705882353,
 'DT_test_f1': 0.94,
 'DT_test_gmean': 0.9553120086485319,
 'RF_test_gmean': 0.970530479565019,
 'DT_test_balanced_accuracy': 0.9399509803921569,
 'RF_test_balanced_accuracy': 0.9607843137254902,
 'x_test': array([[5.1, 3.5, 1.4, 0.2],
        [4.8, 3. , 1.4, 0.1],
        [4.3, 3. , 1.1, 0.1],
        [5.4, 3.9, 1.3, 0.4],
        [5.1, 3.5, 1.4, 0.3],
        [5.7, 3.8, 1.7, 0.3],
        [4.6, 3.6, 1. , 0.2],
        [5.1, 3.3, 1.7, 0.5],
        [4.8, 3.4, 1.9, 0.2],
        [5.2, 3.4, 1.4, 0.2],
        [5.2, 4.1, 1.5, 0.1],
        [4.9, 3.1, 1.5, 0.1],
        [5.5, 3.5, 1.3, 0.2],
        [4.4, 3. , 1.3, 0.2],
     

In [24]:
from copy import deepcopy
from collections import defaultdict
from reduction.classification import SubspaceRulesClassifier
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.model_selection import StratifiedKFold, KFold, ShuffleSplit, RepeatedKFold, RepeatedStratifiedKFold, StratifiedShuffleSplit
from imblearn.metrics import geometric_mean_score, sensitivity_score, specificity_score
from sklearn.metrics import recall_score, make_scorer, confusion_matrix, f1_score, precision_score, balanced_accuracy_score, cohen_kappa_score
from reduction.rule_measures import BayesianRuleMeasures, covered_by_statements
from sympy import parse_expr, evalf

In [25]:
from scipy.stats import entropy
classes_count = len(np.unique(y_train))

def calculate_entropy(y, classes_count):
    counts = np.unique(y, return_counts=True)[1]
    return entropy(counts, base=classes_count)

In [26]:
def accuracy_with_rf(estimator, X, y):
    y_rf = clf_rf.predict(X)
    y_model = estimator.predict(X)
    
    return accuracy_score(y_rf, y_model)

In [27]:
def bal_accuracy_with_rf(estimator, X, y):
    y_rf = clf_rf.predict(X)
    y_model = estimator.predict(X)
    
    return balanced_accuracy_score(y_rf, y_model)

In [28]:
def rf_kohen_cappa(estimator, X, y):
    y_rf = clf_rf.predict(X)
    y_model = estimator.predict(X)
    
    return cohen_kappa_score(y_rf, y_model)

In [29]:
def get_val(clique, x_train, y_train, x_test, y_test):
    
    rules = np.array(all_rules)[clique]
    clf = SubspaceRulesClassifier(rules=rules, max_depth=MAX_DEPTH, random_state=42)
    
    all_scores = defaultdict(list)
    all_additional_scores = defaultdict(list)
    skf = ShuffleSplit(n_splits=CV, test_size=0.5,random_state=42)
    with joblib.parallel_backend('threading'):
        scores = cross_validate(clf, x_train, y_train, n_jobs=1, scoring={
            'balanced_accuracy': 'balanced_accuracy',
            'f1': 'f1_weighted',
            'accuracy': 'accuracy',
            'g_mean': make_scorer(geometric_mean_score, average='weighted'),
            'recall': 'recall_weighted',
            'precision': 'precision_weighted',
            'rf_accuracy': accuracy_with_rf,
            'rf_balanced_accuracy': bal_accuracy_with_rf,
            'rf_cohen_kappa': rf_kohen_cappa
        }, cv=skf)

    additional_scores = defaultdict(list)

    skf = ShuffleSplit(n_splits=CV, test_size=0.5, random_state=42)
    for train_index, test_index in skf.split(x_train, y_train):
        x_train_split = x_train[train_index]
        y_train_split = y_train[train_index]

        mean_rule_scores = defaultdict(list)
        for rule in rules:
            covered_indicies = list(covered_by_statements(rule, x_train_split))

            rule_measurements = BayesianRuleMeasures.create(rule, x_train_split, y_train_split)
            mean_rule_scores['a'].append(rule_measurements.a())
            mean_rule_scores['b'].append(rule_measurements.b())
            mean_rule_scores['c'].append(rule_measurements.c())
            mean_rule_scores['d'].append(rule_measurements.d())
            mean_rule_scores['s'].append(rule_measurements.s_measure())
            mean_rule_scores['n'].append(rule_measurements.n_measure())
            mean_rule_scores['entropy'].append(calculate_entropy(y_train_split[covered_indicies], classes_count))

        for score_name, this_score in dict(mean_rule_scores).items():
            additional_scores[score_name].append(np.mean(this_score))


            
    final_clf = SubspaceRulesClassifier(rules=rules, max_depth=MAX_DEPTH, random_state=42)
    final_clf.fit(x_train, y_train)
    final_clf_y_pred = final_clf.predict(x_test)
    final_model_test_accuracy = accuracy_score(y_test, final_clf_y_pred) 
    final_model_confusion_matrix = confusion_matrix(y_test, final_clf_y_pred)
    
    scores = {
        **scores,
        **dict(additional_scores),
    }
    scores_without_test_preffix = {
        **{k[5:] if k.startswith('test_') else k: np.mean(v) for k, v in scores.items()},
        **{f"{k[5:]}_stddev" if k.startswith('test_') else f"{k}_stddev": np.std(v) for k, v in scores.items()},
    }
    
    score_by_selection_method = {
        'score ' + method: float(parse_expr(method).evalf(subs=scores_without_test_preffix)) for method in SELECTION_METHODS
    }
    
    return {
        **scores_without_test_preffix, 
        **score_by_selection_method,
        'final_model_test_accuracy': final_model_test_accuracy,
        'final_model_test_predictions': final_clf_y_pred,
        'final_model_confusion_matrix': final_model_confusion_matrix,
        'final_model_used_trees': len(final_clf._clf_by_rule),
        'final_model_test_f1_score': f1_score(y_test, final_clf_y_pred, average='weighted'),
        'final_model_test_g_mean': geometric_mean_score(y_test, final_clf_y_pred, average='weighted'),
        'final_model_test_recall_score': recall_score(y_test, final_clf_y_pred, average='weighted'),
        'final_model_test_precision_score': precision_score(y_test, final_clf_y_pred, average='weighted'),
        'final_model_test_balanced_accuracy_score': balanced_accuracy_score(y_test, final_clf_y_pred),
        'final_model_original_rf_fidelity': accuracy_score(rf_test_predict, final_clf_y_pred)
    }

In [30]:
import warnings
warnings.filterwarnings('ignore') 

In [31]:
# import contextlib
# import joblib
# from tqdm.notebook import tqdm    
# from joblib import Parallel, delayed

# @contextlib.contextmanager
# def tqdm_joblib(tqdm_object):
#     """Context manager to patch joblib to report into tqdm progress bar given as argument"""
#     class TqdmBatchCompletionCallback(joblib.parallel.BatchCompletionCallBack):
#         def __init__(self, *args, **kwargs):
#             super().__init__(*args, **kwargs)

#         def __call__(self, *args, **kwargs):
#             tqdm_object.update(n=self.batch_size)
#             return super().__call__(*args, **kwargs)

#     old_batch_callback = joblib.parallel.BatchCompletionCallBack
#     joblib.parallel.BatchCompletionCallBack = TqdmBatchCompletionCallback
#     try:
#         yield tqdm_object
#     finally:
#         joblib.parallel.BatchCompletionCallBack = old_batch_callback
#         tqdm_object.close()    


In [32]:
import time
scoring_start = time.time()

In [33]:
%%time
with joblib.parallel_backend(parallel_backend):
#     with tqdm_joblib(tqdm(desc="My calculation", total=len(subspaces_to_check))) as progress_bar:
    score_by_subspace = \
        dict(zip(
            map(tuple)(subspaces_to_check), 
            Parallel(n_jobs=N_JOBS)(delayed(lambda subspace: get_val(subspace, x_train, y_train, x_test, y_test))(subspace) for subspace in subspaces_to_check)
        ))


CPU times: user 7.8 s, sys: 602 ms, total: 8.4 s
Wall time: 7.91 s


In [35]:
# score_by_subspace_as_list = list(score_by_subspace.items())

In [36]:
# vectorized_results = np.array([
#     [
#         scores['balanced_accuracy'],
#         scores['f1'],
#         scores['accuracy'],
#         scores['g_mean'],
#         scores['recall'],
#         scores['precision'],
#         scores['a'],
#         scores['b'],
#         scores['c'],
#         scores['d'],
#         scores['s'],
#         scores['n']
        
#     ] for scores in score_by_subspace.values()
# ])

In [37]:
# def closest_node(node, nodes):
#     nodes = np.asarray(nodes)
#     dist_2 = np.sum((nodes - node)**2, axis=1)
#     return np.argmin(dist_2)

In [38]:
# closest_node(np.array(current_value), vectorized_results)

In [39]:
# current_value = [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 5, 5, 10, 10, 0.5, 0.5]
# for rules, scores in score_by_subspace.items():
    

In [40]:
# import numpy as np
# from pymoo.model.problem import Problem

# class MyProblem(Problem):

#     def __init__(self):
#         super().__init__(n_var=12,
#                          n_obj=4,
#                          n_constr=0,
#                          xl=np.array([0,0,0,0,0,0,0,0,0,0,0,0]),
#                          xu=np.array([1,1,1,1,1,1, len(x_train), len(x_train), len(x_train), len(x_train), 1, 1]))

#     def _evaluate(self, X, out, *args, **kwargs):
#         closest_points = [
#             closest_node(x, vectorized_results) for x in X
#         ]
        
#         f1 = np.array([
#             1 - score_by_subspace_as_list[point][1]['recall'] for point in closest_points
#         ])
#         f2 = np.array([
#             1 - score_by_subspace_as_list[point][1]['precision'] for point in closest_points
#         ])
#         f3 = np.array([
#             1 - score_by_subspace_as_list[point][1]['g_mean'] for point in closest_points
#         ])
#         f4 = np.array([
#             1 - score_by_subspace_as_list[point][1]['balanced_accuracy'] for point in closest_points
#         ])
        
#         out["F"] = np.column_stack([f1, f2, f3, f4])
# #         out["G"] = np.column_stack([g1, g2])


# vectorized_problem = MyProblem()



In [41]:
# from pymoo.algorithms.nsga2 import NSGA2
# from pymoo.factory import get_sampling, get_crossover, get_mutation
# from pymoo.factory import get_termination
# from pymoo.optimize import minimize

# algorithm = NSGA2(
#     pop_size=100,
#     n_offsprings=20,
#     sampling=get_sampling("real_random"),
#     crossover=get_crossover("real_sbx", prob=0.9, eta=15),
#     mutation=get_mutation("real_pm", eta=20),
#     eliminate_duplicates=True
# )


# termination = get_termination("n_gen", 120)

In [42]:
# res = minimize(vectorized_problem,
#                algorithm,
#                termination,
#                seed=1,
#                save_history=True,
#                verbose=True)

In [43]:
# %%time
# scoring_start = time.time()
# score_by_subspace = {
#     tuple(subspace): get_val(subspace, x_train, y_train, x_test, y_test) for subspace in tqdm(subspaces_to_check)
# }


In [44]:
scoring_end = time.time()
scoring_time = scoring_end - scoring_start

In [45]:
top_rules = max(score_by_subspace, key=lambda s: score_by_subspace[s]['final_model_test_accuracy'])

In [46]:
top = score_by_subspace[top_rules]

In [47]:
scores_by_selection_method = {}
for selection_method in SELECTION_METHODS:
    best_score = max([s[f'score {selection_method}'] for s in score_by_subspace.values()])
    best_score_rules = [rules for rules, val in score_by_subspace.items() if val[f'score {selection_method}'] == best_score]
    
    best_score_rules_with_scoring = {
        rules: score_by_subspace[rules] for rules in best_score_rules
    } 
    worst = best_score_rules_with_scoring[min(best_score_rules_with_scoring, key=lambda v: best_score_rules_with_scoring[v]['accuracy'])]
    best = best_score_rules_with_scoring[max(best_score_rules_with_scoring, key=lambda v: best_score_rules_with_scoring[v]['accuracy'])]
    
    scores_by_selection_method[selection_method] = {
        **{f'worst_{k}': v for k, v in worst.items()},
        **{f'best_{k}': v for k, v in worst.items()},
        'found': len(best_score_rules)
    }
    

In [48]:
results = {
    **meta,
    'scoring_time': scoring_time,
    **scores_by_selection_method,
    **{f'top_{k}': v for k, v in top.items()},
    'found_rules': len(best_score_rules_with_scoring),
    'all_subspaces_with_score': score_by_subspace
}

In [49]:
if PARALLELISM == 'dask':
    client.close()

In [50]:
with open(SAVE_RESULTS_PATH, 'wb') as file:
    pickle.dump(results, file)

FileNotFoundError: [Errno 2] No such file or directory: 'workdir/results/res1-new.results'

In [ ]:
from joblib.externals.loky import get_reusable_executor
get_reusable_executor().shutdown(wait=True)